 # Исследование надёжности заёмщиков

На основе статистики о платёжеспособности клиентов надо было исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.


## Описание данных

**children** — количество детей в семье

**days_employed** — общий трудовой стаж в днях

**dob_years** — возраст клиента в годах

**education** — уровень образования клиента

**education_id** — идентификатор уровня образования

**family_status** — семейное положение

**family_status_id** — идентификатор семейного положения

**gender** — пол клиента

**income_type** — тип занятости

**debt** — имел ли задолженность по возврату кредитов

**total_income** — ежемесячный доход

**purpose** — цель получения кредита

## Вопросы исследования:

Есть ли зависимость между наличием детей и возвратом кредита в срок?

Есть ли зависимость между семейным положением и возвратом кредита в срок?

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Как разные цели кредита влияют на его возврат в срок?

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

Изучим данные, предоставленные банком для исследования, импортируем Pandas, прочитаем файл data.csv, сохраним его в переменной solvency, получим первые 15 строк таблицы.

In [1]:
import pandas as pd # импортируем Pandas
solvency = pd.read_csv('') # читаем файл с данными
solvency.head(15) # выводим первые 15 строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


При первоначальном знакомстве с предоставленными данными сразу возникает вопрос о корректности данных в столбце **days_employed** — общий трудовой стаж в днях (отрицательные значения, слишком большие значения (340266.072047 дней=932 года)), но так как для решения поставленных вопросов эти данные не требуются, оставим их пока без изменений. 
Возможно ошибки возникли в результате использования различных единиц измерения стажа (не дни, а часы, месяцы, годы). 

В столбце education - уровень образования клиента записаны с использованием разных регистров букв, чтобы учесть такие дубликаты, все символы в строке нужно будет привести к нижнему регистру.

В столбце purpose одинаковые цели получения кредита описаны разными конструкциями, необходимо будет лемматизировать данные этого столбца, для получения уникальных значений

Получим общую информацию о данных таблицы solvency с помощью метода info()

In [2]:
solvency.info() # выводим информацию о таблице

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Количество значений в столбцах различается. Это говорит о том, что в столбцах **days_employed** и **total_income** есть пропущенные значения.

Тип данных в столбцах **days_employed** и **total_income** вещественный, стоит заменить его на целочисленный, так как указанный тип точности нам не нужен. 

Посмотрим основные статистические характеристики таблицы, для этого используем метод describe()

In [3]:
solvency.describe() # выводим основные статистические характеристики solvency

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


 В этой таблице необходимо обратить внимание на следующие факты: 1. Минимальное значение в столбце  children равно -1 (вероятно это или ошибка при вводе числа "1", или отсутствующая информация о наличии детей) 2. Максимальное значение в столбце  children равно 20 (вероятно это или ошибка при вводе числа "2", или это не ошибка, но необходимо проверить сколько строк с этим значением) 3. Минимальное значение в столбце  dob_years равно 0 (вероятно это отсутствующая информация о возрасте клиента)

In [4]:
solvency[solvency['children'] == -1].count()[0] #проверяем количество строк со значением -1 в колонке children

47

In [5]:
solvency[solvency['children'] == 20].count()[0] #проверяем количество строк со значением 20 в колонке children

76

In [6]:
solvency[solvency['dob_years'] == 0].count()[0] #проверяем количество строк со значением 0 в колонке dob_years

101

### Вывод

Две проблемы, которые нужно решать: пропуски и некорректно заполненные данные. 

Так как информация хранящаяся в столбце days_employed не является необходимой для решения поставленных вопросов, для заполнения пропусков предположим, что отрицательные значения появились в результате ошибки и все значения в этом столбце приведем к положительным, а пропуски заполним медианой, так как максимальное значение равно 401755.400475 (значительный выброс).

Пропуски в столбце total_income логично заполнить средними значениями дохода по каждому типу занятости.

Для ответов на поставленные перед нами вопросы особенно ценны столбцы  family_status, children, debt, purpose.

Количество строк с числом детей, которое вызывает вопросы: 47+76 = 123, что составит 0.57% от общего числа данных 21525, поэтому предполагаем, что можно заменить эти значения на маркер -1. Это позволит понять при дальнейшем анализе, что эта часть данных на которые опираться не стоит.

Нулевые значения в столбце dob_years логично заполнить средними значениями возраста по каждому типу занятости, несмотря на то что информация хранящаяся в этом столбце возможно не понадобится для ответов на поставленные вопросы.  

Необходимо привести к единому нижнему регистру столбец education, для того чтобы обработать все возможные дубликаты.


### Шаг 2. Предобработка данных

### Обработка пропусков

Проверим данные на наличие пропусков вызовом набора методов для суммирования пропущенных значений.

In [7]:
solvency.isnull().sum() # выводим информацию о пропущенных значениях в таблице

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Для заполнения пропусков предположим, что отрицательные значения появились в результате ошибки и все значения в этом столбце приведем к положительным, а пропуски заполним медианой.

In [8]:
solvency['days_employed'] = solvency['days_employed'].abs() #приводим значения 'days_employed'к положительным
solvency.describe() #проверяем, что отрицательных значений больше нет

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,66914.728907,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,139030.880527,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,24.141633,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,927.009265,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,2194.220567,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,5537.882441,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [9]:
solvency['days_employed'] = solvency['days_employed'].fillna(solvency['days_employed'].median())#заполняем 
# пропущенные значения медианой
solvency.isnull().sum() #проверяем, что в столбце 'days_employed' больше нет нулевых значений

children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Пропущенные значения в столбце total_income заменим на среднее значение по ежемесячному доходу по столбцу income_type

In [10]:
solvency['total_income'] = solvency.groupby('income_type')['total_income'].transform(lambda a:a.fillna(a.mean())) # заменяем 
# пропуски на среднее по income_type с помощью группировки, функции transform() и метода fillna() 

In [11]:
solvency.isnull().sum() #убеждаемся что в таблице больше нет пропусков

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Откорректируем в столбцах children и dob_years некорректные значения на которые обратили внимание при ознакомлении с данными.

In [12]:
solvency['children'] = solvency['children'].replace(20, -1) #заменяем значение 20 на маркер -1

In [13]:
solvency['dob_years'] = solvency.groupby('income_type')['dob_years'].transform(lambda a:a.replace(0, int(a.mean()))) #заменяем 
# нулевые значения средними значениями возраста по каждому типу занятости с помощью группировки, функции transform() 
# и метода replace() 
solvency.describe() # проверим, изменения значений минимального возраста и максимального количества детей

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,21525.000000,2.152500e+04
mean,0.464762,60378.032733,43.495238,0.817236,0.972544,0.080883,1.673959e+05
std,0.757865,133257.558514,12.230322,0.548138,1.420324,0.272661,9.790695e+04
min,-1.000000,24.141633,19.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,1025.608174,34.000000,1.000000,0.000000,0.000000,1.077982e+05
50%,0.000000,2194.220567,43.000000,1.000000,0.000000,0.000000,1.519313e+05
75%,1.000000,4779.587738,53.000000,1.000000,1.000000,0.000000,2.024175e+05
max,5.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [14]:
solvency.info() #убеждаемся что в таблице больше нет пропусков

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Все пропущенные значения в таблице заполненны, некорректные значения в столбцах children и dob_years откорректированы.


### Замена типа данных

Переведём значения столбцов total_income и days_employed в целочисленные значения.

Используем метод astype(), так как изначально указанные столбцы имеют тип float64.

In [15]:
solvency['total_income'] = solvency['total_income'].astype('int') # заменим тип данных в столбце total_income
solvency['days_employed'] = solvency['days_employed'].astype('int') # заменим тип данных в столбце days_employed
solvency.info() #проверим что замена прошла корректно 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


In [16]:
solvency.head() # посмотрим как выглядит преобразованная таблица

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


### Вывод

Перевели значения столбцов total_income и days_employed в целочисленные значения


### Обработка дубликатов

При ознакомлением с датафреймом мы обратили внимание на разный регистр букв в столбце education. Приведем все символы в столбце к нижнему регистру используя метод str.lower()

In [17]:
solvency['education'].value_counts() #посчитаем количество уникальных значений в столбце education методом value_counts() 

среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
УЧЕНАЯ СТЕПЕНЬ             1
Ученая степень             1
Name: education, dtype: int64

In [18]:
solvency['education'] = solvency['education'].str.lower() # используя метод str.lower() приведём все символы к нижнему регистру

In [19]:
solvency['education'].value_counts() # убедимся что дублей в столбце education не осталось

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

Необходимо установить наличие дубликатов используя метод duplicated(), который в сочетании с методом sum() 
возвращает количество дубликатов.

In [20]:
solvency.duplicated().sum() # выводим количество дубликатов в датафрейме

71

Учитывая, что относительно всей таблицы их немного, предположим что они появились вследствие сбоя в записи данных (задвоение данных, ошибка оператора) и удалим их используя метод drop_duplicates()

In [21]:
solvency = solvency.drop_duplicates() # удаляем дубликаты из датафрейма

Проверим, что дубликатов не осталось с помощью метода duplicated() в сочетании с методом sum()

In [22]:
solvency.duplicated().sum() # выводим количество дубликатов после преобразования

0

In [23]:
solvency.info() # получим информацию об обновленном датафрейме

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21454 entries, 0 to 21524
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.1+ MB


### Вывод

Установили наличие дубликатов. Удалили и проверили, что все дубликаты удалены.


### Лемматизация

В датафрейме есть столбец purpose в котором одинаковые цели получения кредита описаны разными словами.

Для начала с помощью метода value_counts() выведем список уникальных значений столбца purpose

In [24]:
solvency['purpose'].value_counts() # выводим уникальные значения целей получения кредита

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Ознакомившись со списком можем выделить 4 основных цели получения кредита: операции с недвижимостью (операции с жильём относим к операциям с недвижимостью так как жильё - это тоже недвижимость), приобретение автомобиля, образование и свадьба. Лемматизируем данные о целях кредита с помощью библиотеки pymystem3.

In [25]:
from pymystem3 import Mystem # импортируем библиотеку pymystem3
m = Mystem()

Применим функцию для лемматизации m.lemmatize к каждой строке столбца purpose с помощью метода apply() и получим новый столбец purpose_lemma в котором содержатся списки лемм по каждой строке.  

In [26]:
solvency['purpose_lemma'] = solvency['purpose'].apply(m.lemmatize) # лемматизируем цели получения кредита

In [27]:
from collections import Counter
print(Counter(solvency['purpose_lemma'].sum())) # посчитаем частоту встречаемости по каждой лемме

Counter({' ': 33570, '\n': 21454, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Опишем функцию, которая будет проверять наличие ключевых слов в списке лемм и возвращать уникальную цель кредита.

In [28]:
# описываем функцию для категоризации целей получения кредита

def purpose_group(lemmas_list):
    if 'жилье' in lemmas_list:
        return 'Недвижимость'
    if 'автомобиль' in lemmas_list:
        return 'Автомобиль'
    if 'свадьба' in lemmas_list:
        return 'Свадьба'
    if 'образование' in lemmas_list:
        return 'Образование'
    return 'Недвижимость'

Применим эту функцию к столбцу с леммами purpose_lemma с помощью метода apply() и создадим новый столбец purpose_total в котором будет содержаться итоговая цель кредита

In [29]:
solvency['purpose_total'] = solvency['purpose_lemma'].apply(purpose_group) #применяем функцию для создания столбца итоговых целей кредитов

In [30]:
solvency['purpose_total'].value_counts()  # проверим уникальные значения столбца purpose_total

Недвижимость    10811
Автомобиль       4306
Образование      4013
Свадьба          2324
Name: purpose_total, dtype: int64

### Вывод

Лемматизировав данные столбца purpose получили новый столбец purpose_total в котором описаны основные цели получения кредитов

### Категоризация данных

Для ответов на вопросы исследования необходима категоризацию данных по следующим столбцам:  наличие детей (children), семейное положение (family_status),	уровень дохода (total_income),	цель кредита(purpose). 

Столбец семейное положение уже категоризирован.

Наличие детей разделим на следующие категории: "нет детей" (0 детей); "есть дети" (1 и более ребёнок).

Категории целей кредита выделили при лемматизации данных они указаны в столбце purpose_total: "Недвижимость", "Автомобиль", "Свадьба", "Образование".

Для выделений категорий уровня дохода воспользуемся персентилями.

Узнаем значения столбца уровень дохода (total_income) соответствующие 25%, 50% (это медиана) и 75% с помощью метода describe()

In [31]:
solvency['total_income'].describe()  # выведем информацию о столбце total_income

count    2.145400e+04
mean     1.674316e+05
std      9.806060e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.518870e+05
75%      2.024170e+05
max      2.265604e+06
Name: total_income, dtype: float64

У нас получится 4 категории уровня дохода: "ниже среднего" (меньше 25%), "средний" (25%-50%), "выше среднего" 50%-75% и "высокий" (больше 75%).

Опишем функцию для категоризации данных о наличии детей

In [32]:
 # описываем функцию для категоризации заёмщиков по факту наличия детей

def children_group(children_count):
    if children_count == 0:
        return 'Нет детей'
    if children_count >= 1:
        return 'Есть дети'

Применим эту функцию к столбцу children с помощью метода apply() и создадим новый столбец children_total в котором будет содержаться информация о категории данных по наличию детей

In [33]:
solvency['children_total'] = solvency['children'].apply(children_group) #применяем функцию для создания столбца 
# с категориями заёмщиков по факту наличия детей

In [34]:
solvency['children_total'].value_counts() # выведем уникальные значения столбца children_total

Нет детей    14091
Есть дети     7240
Name: children_total, dtype: int64

Опишем функцию для категоризации данных о уровне дохода

In [35]:
def income_group(income_sum):
    if income_sum <= 103053:
        return 'Ниже среднего'
    if 103053 < income_sum <= 145017:
        return 'Средний'
    if 145017 < income_sum <= 203435:
        return 'Выше среднего'
    return 'Высокий'

Применим эту функцию к столбцу total_income с помощью метода apply() и создадим новый столбец total_income_group в котором будет содержаться информация о категории данных по уровню дохода

In [36]:
solvency['total_income_group'] = solvency['total_income'].apply(income_group) #применяем функцию для создания столбца 
# с категориями заёмщиков по уровню доходов

In [37]:
solvency['total_income_group'].value_counts() # выведем уникальные значения столбца total_income_group

Выше среднего    6553
Средний          5224
Высокий          4839
Ниже среднего    4838
Name: total_income_group, dtype: int64

In [38]:
 # описываем функцию для категоризации значений столбца debt

def debt_group(debt_value):
    if debt_value == 1:
        return 'Должник'
    return 'Нет задолженностей'

In [39]:
solvency['debt_group'] = solvency['debt'].apply(debt_group) #применяем функцию для создания столбца с категориями надёжности заёмщиков

In [40]:
solvency['debt_group'].value_counts() # выведем уникальные значения столбца debt_group

Нет задолженностей    19713
Должник                1741
Name: debt_group, dtype: int64

### Вывод

Категоризовали датафрейм по следующим данным: уровень дохода, наличие детей в семье, наличие задолженностей по кредиту. По семейному положению категоризация уже была. По целям кредита категоризацию сделали ранее.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на этот вопрос создадим сводную таблицу solvency_pivot_children, используя метод pivot_table()

In [41]:
solvency_pivot_children = solvency.pivot_table(index = ['children_total'], columns = 'debt_group', values = 'debt', aggfunc = 'count')

Добавим в сводную таблицу столбец с информацией об общем количестве заёмщиков по каждой категории

In [42]:
solvency_pivot_children['Всего заёмщиков'] = solvency_pivot_children['Должник'] + solvency_pivot_children['Нет задолженностей']

Выведем "Коэффицент должника" в процентах по каждой категории, он показывает отношение количества заёмщиков имеющих просрочки по оплатам кредитов к общему числу заёмщиков по категории.  

In [43]:
solvency_pivot_children ['Коэффициент Должников'] = solvency_pivot_children['Должник'] / solvency_pivot_children['Всего заёмщиков']

In [44]:
pd.options.display.float_format = '{:,.2%}'.format # скажем Pandas, что бы он перевёл вещественные значения в проценты с 2 знаками после запятой

Отсортируем таблицу по "Коэффициенту Должников", чем меньше коэффициент, тем более "надёжная" категория заёмщиков

In [45]:
solvency_pivot_children.sort_values(by='Коэффициент Должников')

debt_group,Должник,Нет задолженностей,Всего заёмщиков,Коэффициент Должников
children_total,,,,
Нет детей,1063,13028,14091,7.54%
Есть дети,669,6571,7240,9.24%


### Вывод

Из таблицы следует, что люди **не имеющие детей** более ответственно относятся к оплате кредитов в срок

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создаем сводную таблицу по категориям "семейного положения" заёмщиков с помощью метода pivot_table()

In [46]:
solvency_pivot_family_status = solvency.pivot_table(index = ['family_status'], columns = 'debt_group', values = 'debt', aggfunc = 'count')

Добавим в сводную таблицу столбец с информацией об общем количестве заёмщиков по каждой категории

In [47]:
solvency_pivot_family_status['Всего заёмщиков'] = solvency_pivot_family_status['Должник'] + solvency_pivot_family_status['Нет задолженностей']

Выведем "Коэффицент должника" в процентах по каждой категории, он показывает отношение количества кредиторов имеющих просрочки по оплатам кредитов к общему числу заёмщиков по категории.  

In [48]:
solvency_pivot_family_status['Коэффициент Должников'] = solvency_pivot_family_status['Должник'] / solvency_pivot_family_status['Всего заёмщиков']

Отсортируем таблицу по "Коэффициенту Должников", чем меньше коэффициент, тем более "надёжная" категория заёмщиков

In [49]:
solvency_pivot_family_status.sort_values(by='Коэффициент Должников')

debt_group,Должник,Нет задолженностей,Всего заёмщиков,Коэффициент Должников
family_status,,,,
вдовец / вдова,63,896,959,6.57%
в разводе,85,1110,1195,7.11%
женат / замужем,931,11408,12339,7.55%
гражданский брак,388,3763,4151,9.35%
Не женат / не замужем,274,2536,2810,9.75%


### Вывод

Из таблицы следует, что заёмщики **состоящие (или состоявшие) в официальном браке** более ответсвенно относятся к своевременным выплатам по кредитам 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Создаем сводную таблицу по категориям "уровня дохода" заёмщиков с помощью метода pivot_table()

In [50]:
solvency_pivot_total_income = solvency.pivot_table(index = ['total_income_group'], columns = 'debt_group', values = 'debt', aggfunc = 'count')

Добавим в сводную таблицу столбец с информацией об общем количестве заёмщиков по каждой категории

In [51]:
solvency_pivot_total_income['Всего заёмщиков'] = solvency_pivot_total_income['Должник'] + solvency_pivot_total_income['Нет задолженностей']

Выведем "Коэффицент должника" в процентах по каждой категории, он показывает отношение количества заёмщиков имеющих просрочки по оплатам кредитов к общему числу заёмщиков по категории. 

In [52]:
solvency_pivot_total_income['Коэффициент Должников'] = solvency_pivot_total_income['Должник'] / solvency_pivot_total_income['Всего заёмщиков']

Отсортируем таблицу по "Коэффициенту Должников", чем меньше коэффициент, тем более "надёжная" категория заёмщиков

In [53]:
solvency_pivot_total_income.sort_values(by='Коэффициент Должников')

debt_group,Должник,Нет задолженностей,Всего заёмщиков,Коэффициент Должников
total_income_group,,,,
Высокий,341,4498,4839,7.05%
Ниже среднего,383,4455,4838,7.92%
Выше среднего,561,5992,6553,8.56%
Средний,456,4768,5224,8.73%


### Вывод

Заёмщики имеющие уровень дохода **"ниже среднего"** или **"высокий"** являются более надёжными клиентами, чем заёмщики с уровнем дохода "средний" и "выше среднего"

- Как разные цели кредита влияют на его возврат в срок?

Создаем сводную таблицу по категориям "цели получения кредита" заёмщиков с помощью метода pivot_table()

In [54]:
solvency_pivot_purpose_total = solvency.pivot_table(index = ['purpose_total'], columns = 'debt_group', values = 'debt', aggfunc = 'count')

Добавим в сводную таблицу столбец с информацией об общем количестве заёмщиков по каждой категории

In [55]:
solvency_pivot_purpose_total['Всего заёмщиков'] = solvency_pivot_purpose_total['Должник'] + solvency_pivot_purpose_total['Нет задолженностей']

Выведем "Коэффицент должника" в процентах по каждой категории, он показывает отношение количества заёмщиков имеющих просрочки по оплатам кредитов к общему числу заёмщиков по категории. 

In [56]:
solvency_pivot_purpose_total['Коэффициент Должников'] = solvency_pivot_purpose_total['Должник'] / solvency_pivot_purpose_total['Всего заёмщиков']

Отсортируем таблицу по "Коэффициенту Должников", чем меньше коэффициент, тем более "надёжная" категория заёмщиков

In [57]:
solvency_pivot_purpose_total.sort_values(by='Коэффициент Должников')

debt_group,Должник,Нет задолженностей,Всего заёмщиков,Коэффициент Должников
purpose_total,,,,
Недвижимость,782,10029,10811,7.23%
Свадьба,186,2138,2324,8.00%
Образование,370,3643,4013,9.22%
Автомобиль,403,3903,4306,9.36%


### Вывод

Клиенты, которые берут кредит для проведения **операций с недвижимостью** более отвественно относятся к его погашению.

### Шаг 4. Общий вывод

Отвечая на поставленный в описании проекта вопрос: влияет ли семейное положение и наличие детей на факт погашения кредита в срок, можно отметить следующее - **да влияет**: заёмщики не имеющие детей и состоящие (или состоявшие) в официальном браке более ответственно относятся к выплате кредита.

Если описать портрет **"идеального" заёмщика**, то это человек не имеющий детей, состоящий (или состоявший) в официальном браке, имеющий доход "ниже среднего" или "высокий" и берущий кредит для осуществления операций с недвижимостью.

**Менеее надёжный заёмщик** - имеет детей, не состоит в узаконенных отношениях, с уровнем дохода "средний" или "выше среднего" и берёт кредит на приобретения автомобиля.